### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In [198]:
import json         # import pandas.io.json library
import pandas as pd # import pandas library
from datetime import datetime
import os

In [199]:
# load json file into a list
records = [json.loads(line) for line in open("usa.gov_click_data.json")]

#convert the list to a dataframe
data = pd.json_normalize(records)

In [200]:
# getting web browser and operating systems information

data['web_browser'] = data['a'].str.split(" ", 2, expand = True)[0]   # Web Browser details
data['operating_sys'] = data['a'].str.split(" ", 2, expand = True)[1] # Operating Systems info

In [201]:
data['web_browser'].value_counts()[:10]

Mozilla/5.0                 2594
Mozilla/4.0                  601
GoogleMaps/RochesterNY       121
Opera/9.80                    34
TEST_INTERNET_AGENT           24
GoogleProducer                21
Mozilla/6.0                    5
BlackBerry8520/5.0.0.681       4
BlackBerry8520/5.0.0.592       3
Dalvik/1.4.0                   3
Name: web_browser, dtype: int64

In [202]:
data['operating_sys'].value_counts()[:10]

(Windows            1962
(Macintosh;          355
(Windows;            294
(iPhone;             234
(iPad;               170
(Linux;              127
(Ubuntu               37
(BlackBerry;          23
Profile/MIDP-2.1      15
(iPod;                11
Name: operating_sys, dtype: int64

In [203]:
# clear OS data from non-characters
data['operating_sys'] = data['operating_sys'].replace(regex={r';':'',r'\(':'', r'\)':''})

In [204]:
data['operating_sys'].value_counts()[:10]

Windows             2256
Macintosh            356
iPhone               234
iPad                 170
Linux                127
Ubuntu                37
BlackBerry            23
Profile/MIDP-2.1      15
iPod                  12
J2ME/MIDP              9
Name: operating_sys, dtype: int64

In [205]:
# extract the URL the user comes from
data['from_URL'] = data['r'].str.split("\/", expand = True)[2]

# fill the direct links value
data['from_URL'].fillna('direct', inplace = True)

In [206]:
# fill the direct links value
data['from_URL'].fillna('direct', inplace = True)

In [207]:
data['from_URL'].sample(5)

1109                   direct
3226                     t.co
5       www.shrewsbury-ma.gov
101          www.facebook.com
64           www.facebook.com
Name: from_URL, dtype: object

In [208]:
# extract the URL the user is headed to
data['to_URL'] = data['u'].str.split("\/", expand = True)[2]

In [209]:
data['to_URL'].value_counts()[:5]

www.nysdot.gov          836
www.nasa.gov            733
www.whitehouse.gov      169
www.monroecounty.gov    121
www.fda.gov             112
Name: to_URL, dtype: int64

In [210]:
# find the city from which the request is initiated
data['city'] = data['cy']

In [211]:
# track the longitude where the request was sent
data[['longitude', 'latitude']] = data['ll'].apply(pd.Series)

In [212]:
# Retrieve each city time zone
data['time_zone'] = data['tz']

In [213]:
# convert time_in and time_out to local timezone
data['time_in'] = data['hc']
data['time_in'] = pd.to_datetime(data['time_in'], unit = 's').dt.tz_localize('UTC').dt.tz_convert('UTC')

data['time_out'] = data['t']
data['time_out'] = pd.to_datetime(data['time_out'], unit = 's').dt.tz_localize('UTC').dt.tz_convert('UTC')

In [214]:
# drop old data from dataframe
data.drop(columns = ['a', 'c', 'nk', 'tz', 'gr', 'g', 'h', 'l', 'al', 'hh', 'r', 'u', 't',
       'hc', 'cy', 'll', '_heartbeat_', 'kw'], inplace = True)
data.dropna(inplace = True)

In [215]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2752 entries, 0 to 3559
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   web_browser    2752 non-null   object             
 1   operating_sys  2752 non-null   object             
 2   from_URL       2752 non-null   object             
 3   to_URL         2752 non-null   object             
 4   city           2752 non-null   object             
 5   longitude      2752 non-null   float64            
 6   latitude       2752 non-null   float64            
 7   time_zone      2752 non-null   object             
 8   time_in        2752 non-null   datetime64[ns, UTC]
 9   time_out       2752 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(2), object(6)
memory usage: 236.5+ KB


In [216]:
data

,web_browser,operating_sys,from_URL,to_URL,city,longitude,latitude,time_zone,time_in,time_out
0,Mozilla/5.0,Windows,www.facebook.com,www.ncbi.nlm.nih.gov,Danvers,42.576698,-70.954903,America/New_York,2012-03-15 14:48:38+00:00,2012-03-16 18:40:47+00:00
2,Mozilla/4.0,Windows,t.co,boxer.senate.gov,Washington,38.900700,-77.043098,America/New_York,2012-03-16 17:45:41+00:00,2012-03-16 18:40:50+00:00
3,Mozilla/5.0,Macintosh,direct,apod.nasa.gov,Braz,-23.549999,-46.616699,America/Sao_Paulo,2012-03-16 18:37:48+00:00,2012-03-16 18:40:49+00:00
4,Mozilla/5.0,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:53:31+00:00,2012-03-16 18:40:51+00:00
5,Mozilla/5.0,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:55:06+00:00,2012-03-16 18:40:52+00:00
...,...,...,...,...,...,...,...,...,...,...
3553,Mozilla/4.0,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:53:31+00:00,2012-03-16 19:40:40+00:00
3554,Mozilla/4.0,Windows,www.shrewsbury-ma.gov,www.shrewsbury-ma.gov,Shrewsbury,42.286499,-71.714699,America/New_York,2010-05-12 13:55:06+00:00,2012-03-16 19:40:40+00:00
3555,Mozilla/4.0,Windows,direct,www.fda.gov,Paramus,40.944500,-74.070000,America/New_York,2011-03-28 07:47:59+00:00,2012-03-16 19:40:41+00:00
3556,Mozilla/5.0,Windows,www.facebook.com,www.okc.gov,Oklahoma City,35.471500,-97.518997,America/Chicago,2011-06-08 10:50:47+00:00,2012-03-16 19:40:44+00:00


In [217]:
print("Rows available:", data.shape[0])

Rows available: 2752


In [218]:
output_path = os.getcwd()

In [219]:
print("File output path:", output_path)

File output path: D:\ITI\Courses\17. Python for Data Science\Material\Task 2


In [220]:
# convert dataframe to a CSV file
data.to_csv('USA Data from Bitly.csv', index = False)